# 분석 템플릿

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook
import seaborn as sns
import pro as y

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [ ]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
plt.rcParams["figure.figsize"] = (8,4)

## 데이터 준비
1. 전체(df_ori) 데이터에서 조건에 맞는 데이터 추출 (df_main)  
    
    1.1 상위 25퍼센트만 남기기(top_25_df)

2. link 크롤링(link_df)  

    2-1. df 와 link 합치기(top_25_link_df)

3. 별점 크롤링(score_df_list)

In [6]:
# df_ori
df_ori = pd.read_csv('./ZMS_200701.csv')
df_ori

,name,category,address,score,eval_cnt,review_cnt,lat,lng,distance,ZMS
0,크래프트아일랜드 강남역점,"호프,요리주점",서초동 1317-32,4.8,173,292,37.498478,127.025597,1401.450583,100.00
1,크래버 대게나라 서초점,"게,대게",서초동 1582-16,4.8,166,101,37.486660,127.010803,870.179020,98.60
2,미즈컨테이너 강남웨스트점,양식,서초동 1316-29,3.4,603,354,37.498270,127.025215,1369.162723,88.51
3,영동족발 3호점,"족발,보쌈",양재동 1-8,3.5,410,105,37.484180,127.038309,1578.216483,85.02
4,마키노차야 역삼점,해산물뷔페,역삼동 824-46,3.6,322,210,37.498140,127.031071,1587.181976,84.38
...,...,...,...,...,...,...,...,...,...,...
1415,명랑핫도그 서초우성점,패스트푸드,서초동 1337-14,1.0,5,2,37.492195,127.029043,973.573941,6.94
1416,로봇김밥 신분당선양재역점,분식,양재동,1.0,4,8,37.473198,127.038170,2138.641337,6.71
1417,배수경양재정육식당 17호점,"육류,고기",양재동 13-13,1.0,5,0,37.482996,127.036419,1443.782964,6.29
1418,파스쿠찌 교대메가스터디점,커피전문점,서초동 1671-2,1.0,4,3,37.493280,127.014626,923.268458,6.23


In [ ]:
# df_main 만들기
df_main =

In [ ]:
# top_25_df 확인
df_main.describe()

In [ ]:
# top_25_df 만들기
top_25_df = 
n_25 = len(top_25_df)-1 # top 25%가 되는 index
top_25_df

In [ ]:
# 드라이버 켜주고 지도위치 설정
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3) # 3초 기다려주자
driver.get('https://map.kakao.com/')

In [ ]:
# 링크 크롤링
name_list = list(top_25_df.name)
names = []
link = []

search = driver.find_element_by_name('q')
bnt_cle = driver.find_element_by_id('info.main.header.clear')

for name in name_list:
    btn_c = driver.find_element_by_css_selector('#search\.keyword\.bounds') 
    btn_c.click()
    search.send_keys(name)
    search.send_keys(Keys.RETURN)
    driver.implicitly_wait(5)
    time.sleep(0.5)
    soup = bs(driver.page_source,'html.parser')

    name_s = soup.select('#info\.search\.place\.list > li > div.head_item.clickArea > strong > a.link_name')
    names += [i.text for i in name_s]
    link_s = soup.select('#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview')
    link += [i.get('href') for i in link_s]
    bnt_cle.click()
link_df = pd.DataFrame({'name':names,'link':link})

In [ ]:
top_25_link_df = pd.merge(top_25_df,link_df,on='name',how='left')
top_25_link_df # 중복확인

In [ ]:
# score_df_list 만들기
score_df_list = []
for row in top_25_link_df.index:
    df = pd.DataFrame(y.yscl2(top_25_link_df.loc[row,'link']),index=['date','score'])
    score_df_list.append(df.T)

In [ ]:
# 잘 되었는지 확인
for i in range(n_25):
    print(i,top_25_link_df.loc[i,'eval_cnt'],len(score_df_list[i]))

In [ ]:
# 전처리
y.score_srt_int(score_df_list,n_25)

## 데이터 살펴보기

1. ZMS 점수 분포 전체와 비교하기(밀도함수)  
  
2. ZMS 점수 통계치 확인  
  
3. 2018 ~ 2020 분기별 별점,별점count 그래프 확인  
  
4. 2018 ~ 2020 분기별 별점count 증감 그래프 확인  

In [ ]:
# 1.
fig, ax =plt.subplots()
ax.set_xlim([0,100])
ax.set_ylim([0,0.04])
ax.set_xticks([i*10 for i in range(0,11)])
df_ori.ZMS.plot.density(label = '전체')
df_main.ZMS.plot.density(label = '분석대상')
plt.legend(loc='best')

In [ ]:
# 2.
df_main.ZMS.describe()

In [ ]:
# 3. 
for a in range(n_25): # 숫자 넣을 것
    y.score_1820(score_df_list,a,top_25_link_df)

In [ ]:
# 4.
for n in range(n_25): # 숫자 넣을 것
    y.score_delta_1820(score_df_list,n,top_25_link_df)
    plt.axhline(y=0, color='k', linewidth=2)

## 결과 내기

1. ZMS 점수와 위 3번 그래프 점수가 좋은 집 확인    

    1.1 ZMS등수와 3번이 다 높은집 // 둘 중 하나가 높은 집 // 

2. 4번 그래프로 이상치(댓글알바 같은) 확인 후 필터링

## 결과물 토의하기

1. 맛집 리스트와 ZMS 점수, 3개년 점수?